## Part 1: Data Processing

### Process both time series and gamma phase-amplitude coupling (GPAC) images into dataframes

In [1]:
import numpy as np
import os
import pandas as pd
import scipy.io as sio

from IPython.display import display

/Users/lucasrea/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/lucasrea/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def process_timeseries_data(path):
    """ Reads and processes all time series data from MATLAB files in the provided
    path. Only relevant data will be stored for use here.
    """
    all_data = {}

    for file_name in os.listdir(path):
        # 'data' will contain all relevant data that should be used for training.
        # Other keys will hold other useful information that shouldn't be directly
        # needed for training.
        data = {'data': {}}
        
        full_file_path = os.path.join(path, file_name)
        mat_data = sio.loadmat(full_file_path)

        # Time series data with size equal to the number of channels (i.e. electrodes).
        # The electrode names can be indexed to find the electrode that a specific
        # value in this time series data belongs to.
        data['data']['samples'] = list(np.swapaxes(np.array(mat_data['Data']), 0, 1))

        # Time points in seconds
        data['data']['time'] = np.array(mat_data['Time'][0])

        data['data'] = pd.DataFrame(data['data'])
        
        # List of electrode names following the 10-20 EEG recording system
        data['electrodes'] = np.array([item.item() for item in mat_data['Electrodes'][0]])

        all_data[full_file_path] = data

    return all_data


def process_gpac_image_data(path):
    """ Reads and processes all GPAC image data from MATLAB files in the provided
    path. Only relevant data will be stored for use here.
    """
    all_data = {}

    for file_name in os.listdir(path):
        # 'data' will contain all relevant data that should be used for training.
        # Other keys will hold other useful information that shouldn't be directly
        # needed for training.
        data = {'data': {}}
        
        full_file_path = os.path.join(path, file_name)
        mat_data = sio.loadmat(full_file_path)

        # Sample point number corresponding to the start of each 2s window
        data['data']['window_start'] = np.array(mat_data['data']['wind'][0][0][0])

        # GPAC data of size lf x hf, where lf and hf are the dimensions of the GPAC image
        # Note that each window has its data flattened to one list of size lf x hf. We need
        # to cast the reshaped data as a list so that, for this column, each row contains a
        # list of the lf x hf data.
        data['data']['fv'] = np.array(mat_data['data']['fv'][0][0])
        data['data']['fv'] = list(data['data']['fv'].reshape(data['data']['fv'].shape[0], -1))

        # Window labels: baseline interictal (0), blink (1), movement (2), or EMG (3). -1
        # means the window wasn't labelled.
        data['data']['label'] = np.squeeze(np.array(mat_data['training_labels']), axis=1)

        data['data'] = pd.DataFrame(data['data'])

        # The duration of the window chosen for analysis, which should always be 2s
        data['window_len'] = np.array(mat_data['ws'][0])
        
        # lf and hf are the frequency values corresponding to the phase range and amplitude
        # range of interest in the GPAC feature, respectively.
        data['lf'] = np.array(mat_data['lf'][0])
        data['hf'] = np.array(mat_data['hf'][0])

        all_data[full_file_path] = data

    return all_data

In [3]:
# Read all relevant data from the MATLAB files
BASE_DATA_PATH = "data"
TIME_SERIES_DATA_PATH = os.path.join(BASE_DATA_PATH, "TimeSeries")
GPAC_IMAGES_DATA_PATH = os.path.join(BASE_DATA_PATH, "GPAC_Images_Labelled")

timeseries_data = process_timeseries_data(TIME_SERIES_DATA_PATH)
gpac_image_data = process_gpac_image_data(GPAC_IMAGES_DATA_PATH)

In [4]:
test_file = 'data_AL_D6_2_1_export.mat'
test_gpac_data = gpac_image_data[os.path.join(GPAC_IMAGES_DATA_PATH, test_file)]
test_timeseries_data = timeseries_data[os.path.join(TIME_SERIES_DATA_PATH, test_file)]

# Time series data
print("---------- TIME SERIES DATA FORMAT ----------")

print("\n\nData For Training:")
display(test_timeseries_data['data'])

print("\n\nElectrode Names:")
display(test_timeseries_data['electrodes'])

print(f"\n\nNumber of Channels Per Sample: {len(test_timeseries_data['data']['samples'][0])}")


# GPAC image data
print("\n\n\n---------- GPAC IMAGE DATA FORMAT ----------")

print("\n\nData For Training:")
display(test_gpac_data['data'])

print(f"\n\nWindow Length (seconds): {test_gpac_data['window_len'].item()}")

print("\n\nPhase Range Of Interest:")
display(test_gpac_data['lf'])

print("\n\nAmplitude Range Of Interest:")
display(test_gpac_data['hf'])

print(f"\n\nNumber of Datapoints Per GPAC Image: {len(test_gpac_data['data']['fv'][0])}")


---------- TIME SERIES DATA FORMAT ----------


Data For Training:


,samples,time
0,"[6.108219724308253e-05, 4.1935847751014113e-05...",0.000
1,"[6.0519399017154524e-05, 4.2618500927900726e-0...",0.002
2,"[6.742417259467301e-05, 4.1556992611811064e-05...",0.004
3,"[5.537121535626189e-05, 4.2360979111764424e-05...",0.006
4,"[5.912841684511108e-05, 4.846190553369728e-05,...",0.008
...,...,...
949494,"[-3.11911321717285e-05, 2.3324337114086137e-05...",1898.988
949495,"[-2.9876694427662773e-05, 2.2369713074451038e-...",1898.990
949496,"[-3.09238426711861e-05, 2.1764404072353062e-05...",1898.992
949497,"[-3.018560838381861e-05, 1.9767057467827266e-0...",1898.994




Electrode Names:


array(['FP1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'F9', 'T9', 'P9',
       'Fz', 'Cz', 'Pz', 'FP2', 'F8', 'T4', 'T6', 'O2', 'F4', 'C4', 'P4',
       'F10', 'T10', 'P10'], dtype='<U3')



Number of Channels Per Sample: 25



---------- GPAC IMAGE DATA FORMAT ----------


Data For Training:


...



Window Length (seconds): 2


Phase Range Of Interest:


array([ 1. ,  1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,
        2.1,  2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,
        3.2,  3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,
        4.3,  4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,
        5.4,  5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,
        6.5,  6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,
        7.6,  7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,
        8.7,  8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,
        9.8,  9.9, 10. ])



Amplitude Range Of Interest:


array([ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150])



Number of Datapoints Per GPAC Image: 11921


In [61]:
def label_time_series(time_series, GPAC_series):
    """
    Fucntion that will translate the labels from the GPAC Dataframe to the Time-Series DataFrame 
    """
    
    # Initial label value for all rows in the time_series
    time_series['data']['label'] = -1
    
    # Iterate through the GPAC rows to obtain the window value and the label 
    for index, row in GPAC_series['data'].iterrows():
        window_start = int(row['window_start'])
        label_value = row['label']
        
        # Set the 'label' value for the time series Dataframe
        # Add 1000 to window_start because in time_series it is separated by intervals of 0.002 s.
        # 1 window size = 2s, 2 / 0.002 = 1000 rows 
        time_series['data'].loc[window_start : window_start+1000, 'label'] = label_value
    
    return time_series
    
test_timeseries_data_labelled = label_time_series(test_timeseries_data, test_gpac_data)

print("\n\nAs seen in the GPAC Dataframe in the previous cell, the label begins at t=1436, and below you can see that change:")
display(test_timeseries_data_labelled['data'].iloc[1433:1439])

print("\n\nSimilarly, the second label begins at t=2436, and below you can see that change:")
display(test_timeseries_data_labelled['data'].iloc[2433:2439])

print("\n\nLabel value statistics of the time-series data:")
print(test_timeseries_data_labelled['data']['label'].value_counts())

print("\n\nLabel value statistics of the GPAC data:")
print(test_gpac_data['data']['label'].value_counts())



As seen in the GPAC Dataframe in the previous cell, the label begins at t=1436, and below you can see that change:


,samples,time,label
1433,"[-3.918726818570923e-05, 3.1590051061025356e-0...",2.866,-1
1434,"[-2.7482217912228493e-05, 3.294411591614816e-0...",2.868,-1
1435,"[-3.871242545568842e-05, 3.251909070927831e-05...",2.870,-1
1436,"[-4.014147248365788e-05, 3.043857439682265e-05...",2.872,0
1437,"[-3.478690402970244e-05, 3.202987604497138e-05...",2.874,0
1438,"[-3.337380150125322e-05, 3.7954873986926406e-0...",2.876,0




Similarly, the second label begins at t=2436, and below you can see that change:


,samples,time,label
2433,"[6.336374464748228e-05, 6.706071296848414e-05,...",4.866,0
2434,"[5.307997547277073e-05, 6.623601097400111e-05,...",4.868,0
2435,"[5.416560886950276e-05, 6.953586163041345e-05,...",4.870,0
2436,"[6.708489008597993e-05, 7.019486785538814e-05,...",4.872,3
2437,"[7.915946937218587e-05, 7.136690139654629e-05,...",4.874,3
2438,"[7.392872574665033e-05, 7.387425424376896e-05,...",4.876,3




Label value statistics of the time-series data:
-1    889499
 3     15000
 2     15000
 1     15000
 0     15000
Name: label, dtype: int64


Label value statistics of the GPAC data:
-1    886
 3     15
 2     15
 1     15
 0     15
Name: label, dtype: int64


In [56]:
def split_GPAC_data():
    """
    Function that takes all of the data, filters out the rows w/ a label of -1, and then splits 
    the data into Training, Validation, and Test 
    """
    
    all_data = pd.DataFrame(data=None, columns=['fv', 'label'])
    for dataset in gpac_image_data:
        
        # Isolate the image data portion of each dataset
        sub_data = gpac_image_data[dataset]['data']
        sub_data = sub_data.drop(['window_start'], axis=1)
        
        # Now we need to filter out the rows where the 'label' value is -1
        df1 = sub_data[sub_data['label'] > -1]
        all_data = all_data.append(df1)
 
    # Now is the splitting of the dataframe into train, test, and validation datafames
    train_data, valid_data, test_data = np.split(all_data.sample(frac=1, random_state=42),
                                                [int(0.7 * len(all_data)), int(0.85 * len(all_data))])

    return train_data, valid_data, test_data

train_GPAC_data, valid_GPAC_data, test_GPAC_data = split_GPAC_data()

print('\n\n\nTraining DataFrame and class breakdown')
display(train_GPAC_data)
print(train_GPAC_data['label'].value_counts())

print('\n\n\nValidation DataFrame and class breakdown')
display(valid_GPAC_data)
print(valid_GPAC_data['label'].value_counts())


print('\n\n\nTesting DataFrame and class breakdown')
display(test_GPAC_data)
print(test_GPAC_data['label'].value_counts())




Training DataFrame and class breakdown


,fv,label
29,"[0.026578952832569915, 0.026412519206571176, 0...",0
78,"[0.02800487592515676, 0.029052029221901535, 0....",1
55,"[0.025124325652987813, 0.031375009659407566, 0...",3
47,"[0.020210080472589032, 0.022508189853962215, 0...",1
122,"[0.030996552869302977, 0.026865984990070035, 0...",0
...,...,...
308,"[0.010803333963981525, 0.0118814992139812, 0.0...",3
64,"[0.031236079506050358, 0.031084527056174267, 0...",2
117,"[0.014191182648672597, 0.015199027755557856, 0...",0
16,"[0.020171488917380204, 0.024004959061443593, 0...",0


1    190
2    183
0    181
3    159
Name: label, dtype: int64



Validation DataFrame and class breakdown


,fv,label
52,"[0.014733216292781329, 0.017871067660787204, 0...",0
415,"[0.01539657580177166, 0.01762881651874175, 0.0...",1
147,"[0.01697825458691218, 0.021063065301587837, 0....",2
830,"[0.029521058016560575, 0.028428084263004413, 0...",2
57,"[0.02071872802952908, 0.023718854449010056, 0....",0
...,...,...
300,"[0.023846782348386927, 0.023604339849121175, 0...",2
80,"[0.022091000048192377, 0.02198952428966747, 0....",0
32,"[0.02298728062890155, 0.022495106975133423, 0....",1
9,"[0.02018893758579846, 0.016546600276505345, 0....",0


3    43
2    39
0    36
1    35
Name: label, dtype: int64



Testing DataFrame and class breakdown


,fv,label
51,"[0.014029436354815213, 0.014280474969277254, 0...",0
370,"[0.03026395156144915, 0.030441104676146632, 0....",1
456,"[0.018051205585293762, 0.020033789914877715, 0...",2
102,"[0.025893012259715704, 0.021900611378626946, 0...",0
36,"[0.019407836535601047, 0.0187561280211469, 0.0...",0
...,...,...
8,"[0.02746254970056796, 0.022406735048318086, 0....",0
107,"[0.02350585332112601, 0.025473906998611087, 0....",2
490,"[0.02423272928835719, 0.02342685552305181, 0.0...",2
28,"[0.015755475703704988, 0.01818003315805917, 0....",3


3    53
0    37
2    33
1    30
Name: label, dtype: int64
